# Prep Weibo

In [55]:
import numpy as np 
import pandas as pd 
import os
import json
import nltk # NLP  
from nltk.corpus import stopwords  
from nltk.stem import WordNetLemmatizer 
from nltk.stem import PorterStemmer 
import re  
import contractions
from tqdm import tqdm

In [13]:
%cd weibo

[WinError 2] The system cannot find the file specified: 'weibo'
c:\Users\willc\Documents\wills ensemble\weibo


In [4]:
seed = 42

np.random.seed(seed)

In [21]:
# set data arrays
id = []
texts = []
y = []
file = open("Weibo.txt")

In [22]:
def get_info(file):
    lines = file.readlines()
    for line in lines:
        sections = line.split("\t")

        label = sections[1][-1]
        if label == "0":
            y.append("false")
        else:
            y.append("true")

        eid = sections[0][4:]
        id.append(eid)
        source_file = f"weibo_tweets/{eid}.json"
        g = open(source_file)
        data = json.load(g)
        g.close()
        texts.append(data[0]["original_text"])

In [23]:
get_info(file)

In [42]:
df = pd.DataFrame([id,texts,y],['id','chinese_text','target']).transpose()
df = df.infer_objects()
df.head()

,id,chinese_text,target
0,10031080900,毛新宇 深情演唱新作《献给爷爷奶奶的歌》 http://t.cn/h1R0Rj,false
1,10031994215,如果没有人相信你，那就自己相信自己；如果没人欣赏你，那就自己欣赏自己；如果没人祝福你，那就自...,false
2,10276391917,看了新闻感慨：大运会火炬，怎么跟烤土豆片儿似的，中间还有彩椒和洋葱 ~,false
3,10313557537,浮躁风气较盛的今天，并不是没有认真做事的人。但是，“认真”的利益选择性和目的导向性，使很多表...,false
4,10402071863,[心] 女孩身上某个器官，爸爸碰两次，男朋友碰一次，老公一次都不能碰。大家猜猜是答案是什么~~,false


In [50]:
from google.cloud import translate_v2 as translate


def translate_text(target: str, text: str) -> dict:
    """Translates text into the target language.

    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """

    translate_client = translate.Client()

    if isinstance(text, bytes):
        text = text.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(text, target_language=target)

    """print("Text: {}".format(result["input"]))
    print("Translation: {}".format(result["translatedText"]))
    print("Detected source language: {}".format(result["detectedSourceLanguage"]))"""

    return result

r = translate_text("en", df["chinese_text"][0])

In [54]:
r

{'translatedText': 'Mao Xinyu affectionately sings the new work &quot;Songs for Grandparents&quot; http://t.cn/h1R0Rj',
 'detectedSourceLanguage': 'zh-CN',
 'input': '毛新宇 深情演唱新作《献给爷爷奶奶的歌》 http://t.cn/h1R0Rj'}

In [57]:
text = []
for i in tqdm(range(len(df))):
    chinese = df["chinese_text"][i]
    english = translate_text("en", chinese)
    text.append(english["translatedText"])
df["text"] = text

100%|██████████| 4664/4664 [3:18:34<00:00,  2.55s/it]  


In [88]:
df["text"] = text

In [58]:
print(len(df))

4664


In [89]:
df.head()

,id,chinese_text,target,text
0,10031080900,毛新宇 深情演唱新作《献给爷爷奶奶的歌》 http://t.cn/h1R0Rj,false,Mao Xinyu affectionately sings the new work &q...
1,10031994215,如果没有人相信你，那就自己相信自己；如果没人欣赏你，那就自己欣赏自己；如果没人祝福你，那就自...,false,"If no one believes in you, then believe in you..."
2,10276391917,看了新闻感慨：大运会火炬，怎么跟烤土豆片儿似的，中间还有彩椒和洋葱 ~,false,"After reading the news, I sighed: Why does the..."
3,10313557537,浮躁风气较盛的今天，并不是没有认真做事的人。但是，“认真”的利益选择性和目的导向性，使很多表...,false,"Today, when impetuousness is prevalent, it doe..."
4,10402071863,[心] 女孩身上某个器官，爸爸碰两次，男朋友碰一次，老公一次都不能碰。大家猜猜是答案是什么~~,false,[Heart] A girl&#39;s father can touch a certai...


In [62]:
# download nltk stuff
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\willc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\willc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\willc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [90]:
# Clean text
import html2text

def remove_punctuations(text):
    return re.sub('\[^]]*\]', '', text)

def remove_characters(text):
    return re.sub('[^a-zA-Z]/g', ' ', text)

def remove_url(text):
    return re.sub(r'http\S+', '', text)

def remove_contractions(text):
    return ' '.join([contractions.fix(word) for word in text.split()])

stop_words = stopwords.words('english')
def remove_stopwords(text):
    return ' '.join([word for word in nltk.word_tokenize(text) if word not in stop_words])
# str(text).split()

#defining the object for Lemmatization
lemmatizer = WordNetLemmatizer()
#defining a function for lemming
def lemmatize_words(text):
    return ' '.join(lemmatizer.lemmatize(word) for word in text.split())

#defining the object for stemming
stemmer = PorterStemmer()
#defining a function for stemming
def stemming_words(text):
    return ' '.join(stemmer.stem(word) for word in text.split())

def convert_to_text(text):
    h = html2text.HTML2Text()
    return h.handle(text)

def clean_text(text):
    text = convert_to_text(text)
    text = remove_url(text)
    text = remove_contractions(text)
    text = text.lower()
    text = remove_punctuations(text)
    text = remove_characters(text)
    text = remove_stopwords(text)
    text = stemming_words(text)
    text = lemmatize_words(text)
    return text

def clean_text2(text):
    text = remove_url(text)
    #text = remove_punctuations(text)
    text = remove_characters(text)

    return text


In [91]:
df2 = df.copy()
df['text'] = df['text'].apply(clean_text)

In [93]:
df.head()
#df.iloc[0].text

,id,chinese_text,target,text
0,10031080900,毛新宇 深情演唱新作《献给爷爷奶奶的歌》 http://t.cn/h1R0Rj,false,mao xinyu affection sing new work `` song gran...
1,10031994215,如果没有人相信你，那就自己相信自己；如果没人欣赏你，那就自己欣赏自己；如果没人祝福你，那就自...,false,"one believ , believ ; one appreci , appreci ; ..."
2,10276391917,看了新闻感慨：大运会火炬，怎么跟烤土豆片儿似的，中间还有彩椒和洋葱 ~,false,"read news , sigh : universiad torch look like ..."
3,10313557537,浮躁风气较盛的今天，并不是没有认真做事的人。但是，“认真”的利益选择性和目的导向性，使很多表...,false,"today , impetu preval , mean peopl work seriou..."
4,10402071863,[心] 女孩身上某个器官，爸爸碰两次，男朋友碰一次，老公一次都不能碰。大家猜猜是答案是什么~~,false,[ heart ] girl 's father touch certain organ b...


In [94]:
# Export dataframe

df.to_csv('..\datasets\weibo.csv')
df2.to_csv('..\datasets\weibo2.csv')

In [71]:
import html2text

h = html2text.HTML2Text()

print(h.handle("Mao Xinyu affectionately sings the new work &quot;Songs for Grandparents&quot; "))

Mao Xinyu affectionately sings the new work "Songs for Grandparents"


